In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
from gensim import corpora, models
from gensim.models import LdaModel, Nmf, LsiModel
from gensim.models.coherencemodel import CoherenceModel
import random
import pickle
from itertools import combinations
import time
from octis.evaluation_metrics.diversity_metrics import TopicDiversity, InvertedRBO
import numpy as np
import IPython.display as ipd
from sklearn.feature_extraction.text import CountVectorizer
from gensim.matutils import Sparse2Corpus

## Data Preparation

In [3]:
with open('./pickled/nctb_tokenized.pickle', 'rb') as f:
    processed_data = pickle.load(f)
    
random.shuffle(processed_data)

In [4]:
len(processed_data)

8650

In [5]:
num_topics = 8

## Vectorization

### BOW

In [6]:
dictionary = corpora.Dictionary(processed_data)
dictionary.filter_extremes(no_below=3, no_above=0.8)
corpus = [dictionary.doc2bow(text) for text in processed_data]

In [7]:
len(dictionary)

25105

### TF-IDF

In [8]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

## Eval Function

In [9]:
def evaluate_topic_words(topic_words, texts, dictionary):
    
    results = {}
    
    if texts and dictionary:

        coherence_c_v = CoherenceModel(topics=topic_words, texts=texts, dictionary=dictionary, coherence='c_v').get_coherence()
        coherence_npmi = CoherenceModel(topics=topic_words, texts=texts, dictionary=dictionary, coherence='c_npmi').get_coherence()
        
        results['coherence_c_v'] = coherence_c_v
        results['coherence_npmi'] = coherence_npmi
    
    octis = {
        'topics': topic_words  
    }
    
    td = TopicDiversity(topk=10)
    irbo = InvertedRBO(topk=10)
    
    results['topic_diversity'] = td.score(octis)
    results['IRBO'] = irbo.score(octis)
    
    return results

## Topic Modelling

In [10]:
def topic_modeling(model, corpus, processed_data, num_topics, dictionary):
    topic_words = []

    start_time = time.time() 
    topic_model = model(corpus=corpus, num_topics=num_topics, id2word=dictionary)
    end_time = time.time()

    runtime = end_time - start_time
    
    topics = topic_model.show_topics(formatted=False)

    for topic in topics:
        words = [item[0] for item in topic[-1]]
        print(f'{topic[0] + 1} : {words}')
        topic_words.append(words)

    results = evaluate_topic_words(topic_words, texts=processed_data, dictionary=dictionary)
    results['runtime'] = runtime
        
    return results

## LDA

In [11]:
lda = topic_modeling(LdaModel, corpus, processed_data, num_topics, dictionary)
lda

1 : ['বাংলা', 'মাটি', 'তড়িৎ', 'সাল', 'ধাতু', 'ফল', 'আন্দোলন', 'পাকিস্তান', 'পাতা', 'চিত্র']
2 : ['দেশ', 'বাংলাদেশ', 'তথ্য', 'সম্পদ', 'ফল', 'উৎপাদন', 'সময়', 'বৃদ্ধি', 'প্রযুক্তি', 'ধরন']
3 : ['বুদ্ধ', 'ফসল', 'বীজ', 'সময়', 'ফল', 'জমি', 'মানচিত্র', 'জাত', 'রা', 'সার']
4 : ['জীবন', 'কথা', 'ঈশ্বর', 'মন', 'আল্লাহ', 'লাভ', 'পড়া', 'অর্থ', 'ধর্ম', 'ব্যক্তি']
5 : ['টাকা', 'ব্যয়', 'হিসাব', 'রাষ্ট্র', 'আয়', 'ব্যাংক', 'মূলধন', 'প্রদান', 'বিক্রয়', 'অর্থ']
6 : ['বাংলাদেশ', 'সরকার', 'স্থান', 'ব্যাংক', 'দেশ', 'সময়', 'পদ্ধতি', 'শিল্প', 'তৈরি', 'গাছ']
7 : ['শিশু', 'সময়', 'বাংলা', 'ভাষা', 'কবি', 'কবিতা', 'মা', 'পৃথিবী', 'দিক', 'পড়া']
8 : ['পানি', 'খাদ্য', 'তৈরি', 'সময়', 'পদার্থ', 'দেহ', 'মাছ', 'পরিমাণ', 'একটা', 'রোগ']


{'coherence_c_v': 0.4800203151190844,
 'coherence_npmi': 0.016172149519098913,
 'topic_diversity': 0.8375,
 'IRBO': 0.9463730266390307,
 'runtime': 2.3936381340026855}

## LDA TF-IDF

In [12]:
lda_model_tfidf = topic_modeling(LdaModel, corpus_tfidf, processed_data, num_topics, dictionary)
lda_model_tfidf

1 : ['খাদ্য', 'বুদ্ধ', 'ভিটামিন', 'রোগ', 'ভিক্ষু', 'প্রোটিন', 'পারমী', 'সঙ্গীতি', 'ইন্টারনেট', 'নাটক']
2 : ['টাকা', 'ব্যাংক', 'সরকার', 'দেশ', 'পরিবার', 'সম্পদ', 'বাংলাদেশ', 'রাষ্ট্র', 'আয়', 'ব্যয়']
3 : ['পানি', 'মাটি', 'উদ্ভিদ', 'চিত্র', 'ফসল', 'খাদ্য', 'মাছ', 'পদার্থ', 'একটা', 'গাছ']
4 : ['কর্তৃক', 'পরিমার্জিত', 'পাঠ্যপুস্তকরূপ', 'বাংলাদেশ', 'অধ্যাপক', 'গণপ্রজাতন্ত্রী', 'নবম', 'দশম', 'বিনামূল্য', 'সহিংসতা']
5 : ['মজিদ', 'মেজ', 'জমিলা', 'কুন্টা', 'রহিমা', 'খোদা', 'রফিক', 'বিএমআ', 'টপোলজি', 'স্যা']
6 : ['নবাব', 'পূজা', 'দেবী', 'পুরাণ', 'এইডস', 'বাজেটিং', 'কার্তিক', 'বাংলা', 'বিহার', 'বয়ঃসন্ধিকাল']
7 : ['তথ্য', 'কম্পিউটার', 'নেটওয়ার্ক', 'উপাত্ত', 'সফটওয়্যা', 'রচনা', 'প্রযুক্তি', 'ধাতু', 'পরিমার্জন', 'যথাসম্ভব']
8 : ['আল্লাহ', 'ঈশ্বর', 'যীশু', 'জীবন', 'হযরত', 'তায়ালা', 'ইসলাম', 'সূ', 'ভালোবাসা', 'মন']


{'coherence_c_v': 0.46609093102037424,
 'coherence_npmi': -0.09202310860491603,
 'topic_diversity': 0.975,
 'IRBO': 0.9955702504688776,
 'runtime': 2.014366865158081}

## LSA

In [13]:
lsa = topic_modeling(LsiModel, corpus, processed_data, num_topics, dictionary)
lsa

1 : ['সময়', 'দেশ', 'বাংলাদেশ', 'ফল', 'পানি', 'তৈরি', 'জীবন', 'সাথে', 'কথা', 'খাদ্য']
2 : ['পানি', 'টাকা', 'ব্যাংক', 'খাদ্য', 'দেশ', 'বাংলাদেশ', 'মাটি', 'সরকার', 'পদার্থ', 'অর্থ']
3 : ['পানি', 'ঈশ্বর', 'কথা', 'ব্যাংক', 'জীবন', 'বাংলাদেশ', 'দেশ', 'টাকা', 'উৎপাদন', 'মন']
4 : ['টাকা', 'বাংলাদেশ', 'দেশ', 'ব্যাংক', 'হিসাব', 'সরকার', 'নগদ', 'পানি', 'ব্যয়', 'অর্থ']
5 : ['আল্লাহ', 'খাদ্য', 'তায়ালা', 'বাংলা', 'সময়', 'টাকা', 'অর্থ', 'কথা', 'একটা', 'পড়া']
6 : ['শিশু', 'আল্লাহ', 'খাদ্য', 'পরিবার', 'তায়ালা', 'মাটি', 'বীজ', 'মা', 'পানি', 'ফসল']
7 : ['বীজ', 'পানি', 'ব্যাংক', 'ফসল', 'শিশু', 'পরিবার', 'মাটি', 'চাষ', 'উৎপাদন', 'জমি']
8 : ['ঈশ্বর', 'ব্যাংক', 'আল্লাহ', 'সৃষ্টি', 'বাংলা', 'খাদ্য', 'তায়ালা', 'যীশু', 'টাকা', 'কুরআন']


{'coherence_c_v': 0.3424405474452942,
 'coherence_npmi': -0.05213085654711766,
 'topic_diversity': 0.4625,
 'IRBO': 0.7744961403413265,
 'runtime': 0.6265265941619873}

## LSA TF-IDF

In [14]:
lsa_model_tfidf = topic_modeling(LsiModel, corpus_tfidf, processed_data, num_topics, dictionary)
lsa_model_tfidf

1 : ['খাদ্য', 'বাংলাদেশ', 'ঈশ্বর', 'দেশ', 'পানি', 'ব্যাখ্যা', 'শিশু', 'পরিবার', 'জীবন', 'ব্যাংক']
2 : ['ঈশ্বর', 'খাদ্য', 'আল্লাহ', 'পানি', 'মাছ', 'ফসল', 'তায়ালা', 'মাটি', 'চাষ', 'পদার্থ']
3 : ['ব্যাংক', 'টাকা', 'হিসাব', 'ঈশ্বর', 'আয়', 'ব্যয়', 'নগদ', 'ব্যবসায়', 'লেনদেন', 'সরকার']
4 : ['টাকা', 'আল্লাহ', 'বাংলাদেশ', 'ঈশ্বর', 'ব্যাংক', 'কর্তৃক', 'তায়ালা', 'হিসাব', 'পরিমার্জিত', 'রচনা']
5 : ['আল্লাহ', 'ঈশ্বর', 'তায়ালা', 'কুরআন', 'সূ', 'আল', 'যীশু', 'হযরত', 'আয়াত', 'হাদিস']
6 : ['টাকা', 'ফসল', 'কৃষি', 'ঈশ্বর', 'বাংলাদেশ', 'বিক্রিয়া', 'চাষ', 'বীজ', 'দেশ', 'তথ্য']
7 : ['খাদ্য', 'ঈশ্বর', 'পরমাণু', 'মাছ', 'বিক্রিয়া', 'রোগ', 'খাবার', 'পরিবেশন', 'ইলেকট্রন', 'ভিটামিন']
8 : ['কর্তৃক', 'পরিমার্জিত', 'ঈশ্বর', 'দশম', 'নবম', 'পাঠ্যপুস্তকরূপ', 'শিশু', 'নির্ধারিত', 'গণপ্রজাতন্ত্রী', 'বিক্রিয়া']


{'coherence_c_v': 0.42619599921718654,
 'coherence_npmi': -0.07058278337720396,
 'topic_diversity': 0.625,
 'IRBO': 0.782523601777296,
 'runtime': 1.31154203414917}

## NMF

In [15]:
nmf = topic_modeling(Nmf, corpus, processed_data, num_topics, dictionary)
nmf

1 : ['শিশু', 'ফল', 'পরিবার', 'দেশ', 'সাল', 'মা', 'রোগ', 'তথ্য', 'সরকার', 'মাছ']
2 : ['ঈশ্বর', 'সৃষ্টি', 'শিশু', 'জীবন', 'যীশু', 'সাথে', 'সময়', 'তৈরি', 'পরিবেশ', 'পৃথিবী']
3 : ['টাকা', 'হিসাব', 'ব্যাংক', 'ব্যয়', 'নগদ', 'সম্পদ', 'সরকার', 'আয়', 'প্রদান', 'লেনদেন']
4 : ['বাংলাদেশ', 'দেশ', 'কথা', 'সমাজ', 'মন', 'জীবন', 'নারী', 'পরিবার', 'রাষ্ট্র', 'ব্যাখ্যা']
5 : ['আল্লাহ', 'ব্যাংক', 'তায়ালা', 'অর্থ', 'হযরত', 'কুরআন', 'দেশ', 'আল', 'পরিবর্তন', 'বাংলাদেশ']
6 : ['বাংলা', 'কথা', 'সময়', 'পড়া', 'মন', 'দিক', 'একটা', 'হাত', 'রাজা', 'শোনা']
7 : ['পানি', 'মাটি', 'দেশ', 'ফসল', 'সময়', 'জমি', 'চাষ', 'উৎপাদন', 'বাংলাদেশ', 'বীজ']
8 : ['খাদ্য', 'পানি', 'তৈরি', 'দেহ', 'পদার্থ', 'পরিমাণ', 'উপাদান', 'মাছ', 'খাবার', 'বিক্রিয়া']


{'coherence_c_v': 0.5392257985109211,
 'coherence_npmi': 0.05514378306832175,
 'topic_diversity': 0.7875,
 'IRBO': 0.939899721702296,
 'runtime': 0.3692286014556885}

## NMF TF-IDF

In [16]:
nmf_tfidf = topic_modeling(Nmf, corpus_tfidf, processed_data, num_topics, dictionary)
nmf_tfidf

1 : ['টাকা', 'ব্যাংক', 'পূজা', 'তাপমাত্রা', 'হিসাব', 'নগদ', 'লেনদেন', 'পৃথিবী', 'ব্যয়', 'দেবী']
2 : ['খাদ্য', 'পানি', 'মাছ', 'ফসল', 'চাষ', 'মাটি', 'পুকুর', 'রোগ', 'খাবার', 'উৎপাদন']
3 : ['তথ্য', 'রচনা', 'পরিমার্জন', 'উপাত্ত', 'তোলা', 'যথাসম্ভব', 'কৃতজ্ঞতা', 'পরিবেশন', 'উদ্দেশ্য', 'সর্বশেষ']
4 : ['শিশু', 'দেশ', 'পরিবেশ', 'বাংলাদেশ', 'খাদ্য', 'শিল্প', 'জনসংখ্যা', 'ব্যবসায়', 'ব্যাখ্যা', 'বুদ্ধ']
5 : ['কর্তৃক', 'পরিমার্জিত', 'বিক্রিয়া', 'পোশাক', 'পাঠ্যপুস্তকরূপ', 'নবম', 'দশম', 'বাংলাদেশ', 'নির্ধারিত', 'সরকার']
6 : ['কবিতা', 'কবি', 'পরমাণু', 'ইলেকট্রন', 'বাংলা', 'সাহিত্য', 'জাতিসংঘ', 'ফুল', 'ভাষা', 'এসিড']
7 : ['আল্লাহ', 'তায়ালা', 'সূ', 'কুরআন', 'আল', 'ইসলাম', 'হযরত', 'হাদিস', 'আয়াত', 'নারী']
8 : ['ঈশ্বর', 'যীশু', 'বস্তু', 'আত্মা', 'বল', 'জীবন', 'পাপ', 'ভর', 'পবিত্র', 'শক্তি']


{'coherence_c_v': 0.518047143842161,
 'coherence_npmi': 0.014686625892125188,
 'topic_diversity': 0.975,
 'IRBO': 0.9953727509790816,
 'runtime': 0.9276916980743408}

In [17]:
duration = 2  
sampling_rate = 44100
frequency = 440.0 
t = np.linspace(0, duration, int(sampling_rate * duration), False)
wave = 0.5 * np.sin(2 * np.pi * frequency * t)

audio = ipd.Audio(wave, rate=sampling_rate, autoplay=True)
display(audio)